In [1]:
#coding=utf-8 
%matplotlib ipympl

import sys
sys.path.insert(0, '../')
from AircraftIden.data_case.GeneralAircraftCase import GeneralAircraftCase, PX4AircraftCase
from AircraftIden import FreqIdenSIMO, TransferFunctionFit
import numpy as np
import matplotlib.pyplot as plt
import math

plt.rc('figure', figsize=(20.0, 10.0))

def show_logs(px4_case: PX4AircraftCase):
    print("LOW")
    needed_data = ['ele', 'q', 'thr', 'body_vx', "iden_start_time"]
    t_arr, data_list = px4_case.get_data_time_range_list(needed_data)
    print(t_arr)
    data_list[-1] = data_list[-1]/10
    plt.figure("Ele")
    plt.grid(which='both')
    plt.plot(t_arr, data_list[0], 'r.', label='ele')

    plt.figure("sourcedata")
    # print(t_arr)
    # print(data_list)
    for i in range(needed_data.__len__()):
        plt.plot(t_arr, data_list[i], label=needed_data[i])
    plt.legend()
    plt.show()


def split_logs(px4_case:PX4AircraftCase):
    needed_data = ["iden_start_time"]
    print("Will start split data with", needed_data)
    t_arr, data_list = px4_case.get_data_time_range_list(needed_data)
    iden_start_time = data_list[-1]
    data_splited = []
    is_in_a_test = False
    for i in range(1, t_arr.__len__()-1):
        if (iden_start_time[i] > iden_start_time[i+1] or (len(data_splited) > 0 and t_arr[i] - data_splited[-1]["start"] > 20)) \
                and is_in_a_test:
            data_splited[-1]["end"] = t_arr[i]
#             print("Data split {}th, {:5.2f}:{:5.2f}  len {:5.2f}".format(
#                 data_splited.__len__(), data_splited[-1]["start"], data_splited[-1]["end"],
#                 data_splited[-1]["end"] - data_splited[-1]["start"]
#             ))
            is_in_a_test = False

        # if (0 <= iden_start_time[i] < iden_start_time[i-1] and iden_start_time[i] < iden_start_time[i+1]):
        if (iden_start_time[i-1] <=0 and 0 < iden_start_time[i]):
            # Is a start
            is_in_a_test = True
            data_splited.append({"start": t_arr[i]})

    return data_splited


def join_data(data_splited, status):
    joined_data_status = {}
    assert data_splited.__len__() == status.__len__(), "Status Length must equal to data_split but {} {}".format(
        data_splited.__len__(), status.__len__())

    for i in range(data_splited.__len__()):
        status_test = status[i]
        if status_test == "-" or status_test == "wrong":
            continue
        if status_test in joined_data_status:
            #Join data
            joined_data_status[status_test].append((data_splited[i]["start"],data_splited[i]["end"]))
        else:
            joined_data_status[status_test] = [(data_splited[i]["start"], data_splited[i]["end"])]
    return joined_data_status

def split_and_join_data(px4_case:PX4AircraftCase,status):
    sp = split_logs(px4_case)
    return join_data(sp,status)


def plot_status_data(test_case, joined_data_status,needed_data):
    for key in joined_data_status:
        plt.figure("Data case: {}".format(key))
        total_time, t_data, datas = test_case.get_concat_data(joined_data_status[key], needed_data)
        for i in range(1, needed_data.__len__()):
            plt.plot(t_data, datas[needed_data[i]], label=needed_data[i])
            pass
        plt.legend()
        plt.grid(which='both')
    plt.show()

def process_status_data(test_case, joined_data_status,omg_min,omg_max,win_num=None):
    needed_data = ['ele', 'q', 'thr', "body_vx", "body_vz","ax","az"]
    res = {}
    for key in joined_data_status:
        total_time, t_data, datas = test_case.get_concat_data(joined_data_status[key], needed_data)
        iden = FreqIdenSIMO(t_data, omg_min, omg_max, datas['ele'], datas['q'], datas["body_vx"], datas["body_vz"],datas["ax"],datas["az"],
                             uniform_input=True, win_num=None)#,assit_input=datas["thr"])
        res[key] = iden
    return res


def plot_idens_for_status(idens):
    for key in idens:
        iden = idens[key]
        plt.figure("ele_q")
        iden.plt_bode_plot(0, label=key)
        plt.figure("ele_body_vx")
        iden.plt_bode_plot(1, label=key)
        plt.figure("ele_body_vz")
        iden.plt_bode_plot(2, label=key)
        plt.figure("ele_body_ax")
        iden.plt_bode_plot(3, label=key)
        plt.figure("ele_body_az")
        iden.plt_bode_plot(4, label=key)
    plt.show()

ModuleNotFoundError: No module named 'pyulog'

In [ ]:
fpath = "/Users/xuhao/Dropbox/FLYLOG/foam-tail-sitter/cruising/log_34_2018-4-10-16-16-04.ulg"
status = [
        "5m/s", "5m/s", "5m/s", "5m/s", "5m/s", "5m/s", "5m/s", "5m/s",
        "8m/s", "8m/s", "8m/s", "8m/s", "8m/s", "8m/s","8m/s", "8m/s",
        "10m/s", "10m/s", "10m/s", "10m/s", "10m/s", "10m/s","10m/s", "10m/s",
]
#process_file(fpath,status)
px4_case = PX4AircraftCase(fpath)
show_logs(px4_case)

status_low = [
        "1m/s", "1m/s", "1m/s", "1m/s", "1m/s", "1m/s", "1m/s", "1m/s",
        "3m/s", "3m/s", "3m/s", "3m/s", "3m/s", "3m/s","3m/s", "3m/s",
        "-", "-", "-",
]

fpath_low = "/Users/xuhao/Dropbox/FLYLOG/foam-tail-sitter/cruising/log_32_2018-4-10-15-53-08.ulg"
px4_case_low = PX4AircraftCase(fpath_low)
show_logs(px4_case_low)


In [ ]:
plt.rc('figure', figsize=(10.0, 5.0))
data_splited = split_logs(px4_case)
data_splited = join_data(data_splited, status)
data_splited_low = split_and_join_data(px4_case_low, status_low)
needed_data = ['ele', 'q', 'thr', "body_vx", "body_vz","ax","az"]
plot_status_data(px4_case, data_splited,needed_data)
plot_status_data(px4_case, data_splited_low,needed_data)

In [2]:
idens = process_status_data(px4_case, data_splited, 5, 50)
idens_low = process_status_data(px4_case_low, data_splited_low, 5, 50)
idens.update(idens_low)

NameError: name 'process_status_data' is not defined

In [17]:
plt.rc('figure', figsize=(20.0, 10.0))
plot_idens_for_status(idens)

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

C:\Users\xuhao\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
..\AircraftIden\FreqIden.py:157: RuntimeWarning: invalid value encountered in log10
  plt.semilogx(freq, 20 * np.log10(gxx), label=label+'gxx')


In [18]:
import pickle
pkl_name = "/Users/xuhao/Dropbox/FLYLOG/foam-tail-sitter/cruising/log_34_2018-4-10-freq-res.pkl"
f = open("/Users/xuhao/Dropbox/FLYLOG/foam-tail-sitter/cruising/log_34_2018-4-10-freq-res.pkl","wb")
pickle.dump(idens, f, pickle.HIGHEST_PROTOCOL)

In [9]:
idens

{'10m/s': <AircraftIden.FreqIden.FreqIdenSIMO at 0x2020a4e4940>,
 '1m/s': <AircraftIden.FreqIden.FreqIdenSIMO at 0x20278a5c278>,
 '3m/s': <AircraftIden.FreqIden.FreqIdenSIMO at 0x2020a4e4f28>,
 '5m/s': <AircraftIden.FreqIden.FreqIdenSIMO at 0x20278a5cd68>,
 '8m/s': <AircraftIden.FreqIden.FreqIdenSIMO at 0x20278a5c2e8>}

In [24]:
pkl_name = "/Users/xuhao/Dropbox/FLYLOG/foam-tail-sitter/cruising/log_34_2018-4-10-freq-res.pkl"
finput = open(pkl_name, 'rb')
idens = pickle.load(finput)
plt.rc('figure', figsize=(20.0, 10.0))
plot_idens_for_status(idens)

C:\Users\xuhao\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
..\AircraftIden\FreqIden.py:157: RuntimeWarning: invalid value encountered in log10
  plt.semilogx(freq, 20 * np.log10(gxx), label=label+'gxx')
